In [1]:
import pandas as pd
import boto3

In [12]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
Key="indicator_loading.csv"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_csv(read_file['Body'],sep=',')

df.fillna('', inplace=True)

In [13]:
df

,Strategy,Mode,Securities,Timeframe,Strategy Parameters
0,simple_strategy_3,backtest,"AUD_USD,EUR_JPY,EUR_USD,GBP_JPY,GBP_USD,NZD_US...",daily,"use_level=yes,use_db=yes"
1,simple_strategy_3,backtest,"AUD_USD,EUR_JPY,EUR_USD,GBP_JPY,GBP_USD,NZD_US...",daily,"use_level=no,use_db=yes"


In [14]:
for i,row in df.iterrows():
            command={}
            command['--strat_name']=row['Strategy']
            command['--mode']=str(row['Mode'])
            command['--tickers']=row['Securities']
            command['--timeframe']=row['Timeframe']
            command['--strat_param']=row['Strategy Parameters']
            final_command='python /usr/local/airflow/dags/q_pack/q_run/run_BT.py '+' '.join([(k+"="+v) for k, v in command.items() if v!=''])
            print(final_command)

python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --strat_name=simple_strategy_3 --mode=backtest --tickers=AUD_USD,EUR_JPY,EUR_USD,GBP_JPY,GBP_USD,NZD_USD,USD_CAD,USD_CHF,USD_JPY --timeframe=daily --strat_param=use_level=yes,use_db=yes
python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --strat_name=simple_strategy_3 --mode=backtest --tickers=AUD_USD,EUR_JPY,EUR_USD,GBP_JPY,GBP_USD,NZD_USD,USD_CAD,USD_CHF,USD_JPY --timeframe=daily --strat_param=use_level=no,use_db=yes


In [9]:
final_commmand

'python /usr/local/airflow/dags/q_pack/q_run/run_BT.py --strat_name=simple_strategy_3 --mode=backtest --tickers=AUD_USD,EUR_JPY,EUR_USD,GBP_JPY,GBP_USD,NZD_USD,USD_CAD,USD_CHF,USD_JPY --timeframe=daily --strat_param=use_level=yes,use_db=yes'

In [1]:
import json
import pandas as pd
import boto3
import io
import numpy as np

from db_pack.zerodha import zerodha_historical
import q_run.run_BT_dynamic as run_BT_dynamic

import q_credentials.db_secmaster_cred as db_secmaster_cred
import psycopg2
db_host=db_secmaster_cred.dbHost 
db_user=db_secmaster_cred.dbUser
db_password=db_secmaster_cred.dbPWD
db_name=db_secmaster_cred.dbName




In [2]:

dag_id = "alpaca_dynamic_new"
universe='US Equity'
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
if universe=='US Equity':
    Key="interested_tickers_alpaca.xlsx"
elif universe=='Forex':
    Key="interested_tickers_oanda.xlsx"
elif universe=='Indian Equity':
    Key="interested_tickers_india.xlsx"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_excel(io.BytesIO(read_file['Body'].read()),sheet_name="d")

ticker_list = list(df['Tickers'])

# Getting the last date for each interested tickers
conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
sql="""select a.min_date, a.max_date, b.id as symbol_id, b.ticker from
    (select min(date_price) as min_date, max(date_price) as max_date, symbol_id
    from {}_data 
    group by symbol_id) a right join symbol b on a.symbol_id = b.id 
    where b.ticker in {} """.format('d',str(tuple(df['Tickers'])).replace(",)", ")"))
df_ticker_last_day=pd.read_sql(sql,con=conn)


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


In [3]:
df_ticker_last_day.to_csv('alpaca_airflow.csv')

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
